### 0. Imports and requirements

* В данном соревновании мы имеем дело с последовательностями, один из интуитивных способов работы с ними - использование рекуррентных сетей. Данный бейзлайн посвящен тому, чтобы показать, как можно строить хорошие решения без использования сложного и трудоемкого feature engineering-а (чтобы эффективно решать ту же задачу с высоким качеством с помощью бустингов нужно несколько тысяч признаков), благодаря рекуррентным сетям. В этом ноутбуке мы построим решение с использованием фреймфорка `torch`. Для комфортной работы Вам понадобится машина с `GPU` (хватит ресурсов `google colab` или `kaggle`).

In [73]:
%load_ext autoreload
%autoreload 2

from sklearn.metrics import roc_auc_score

import plotly
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import Image, display
import os
import pandas as pd
import sys
import pickle
import numpy as np
import torch
import torch.nn as nn
import copy

from sklearn.model_selection import train_test_split
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
pd.set_option('display.max_columns', None)

# добавим корневую папку, в ней лежат все необходимые полезные функции для обработки данных
sys.path.append('../../')
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Data Preprocessing

In [74]:
# dir_path = "../../../Datasets/AlfaBattle2.0/data_for_competition/"
# TRAIN_TRANSACTIONS_PATH = dir_path+'train_transactions_contest/'
# TEST_TRANSACTIONS_PATH = dir_path+'test_transactions_contest/'

# TRAIN_TARGET_PATH = dir_path+'train_target.csv'

In [75]:
TRAIN_TRANSACTIONS_PATH = '../../../train_transactions_contest/'
TEST_TRANSACTIONS_PATH = '../../../test_transactions_contest/'

TRAIN_TARGET_PATH = '../../../train_target.csv'

In [76]:
target_frame = pd.read_csv(TRAIN_TARGET_PATH)
# target_frame.head()

* Как и в случае с бустингами, мы не можем поместить всю выборку в память, в виду, например, ограниченных ресурсов. Для итеративного чтения данных нам потребуется функция `utils.read_parquet_dataset_from_local`, которая читает N частей датасета за раз в память.


* Нейронные сети требуют отделнього внимания к тому, как будут поданы и обработаны данные. Важные моменты, на которые требуется обратить внимание:
    * Использование рекуррентных сетей подразумевает работу на уровне последовательностей, где одна последовательность - все исторические транзакции клиента. Чтобы преобразовать `pd.DataFrame` с транзакциями клиентов в табличном виде к последовательностям, мы подготовили функцию `dataset_preprocessing_utils.transform_transactions_to_sequences`, она делает необходимые манипуляции и возвращает фрейм с двумя колонками: `app_id` и `sequences`. Колонка `sequence` представляет из себя массив массивов длины `len(features)`, где каждый вложенный массив - значения одного конкретного признака во всех транзакциях клиента. 
    
    * каждый клиент имеет различную по длине историю транзакций. При этом обучение сетей происходит батчами, что требует делать паддинги в последовательностях. Довольно неэффективно делать паддинг внутри батча на последовательностях случайной длины (довольно часто будем делать большой и бесполезный паддинг). Гораздо лучше использовать технику `sequence_bucketing` (о ней рассказано в образовательном ролике к данному бейзлайну). Для этого мы предоставляем функцию `dataset_preprocessing_utils.create_padded_buckets`. Один из аргументов в данную функцию - `bucket_info` - словарь, где для конкретной длины последовательности указано до какой длины нужно делать паддинг. Мы предоставялем для старта простой вид разбиения на 100 бакетов и файл где лежит отображение каждой длины в падднг (файл `buckets_info.pkl`).
    
    * Такие признаки, как [`amnt`, `days_before`, `hour_diff`] по своей природе не являются категориальными. Вы в праве самостоятельно выбирать способ работы с ними (модифицируя функции бейзлайна или адаптируя под себя). В рамках бейзлайна мы предлагаем интерпретировать каждую не категориальную фичу как категориальную. Для этого нужно подготовить bin-ы для каждой фичи. Мы предлагаем простой способ разбиения по бинам.

## ALL UTILS FROM .PY

In [77]:
import os
import pandas as pd
import tqdm


def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                     num_parts_to_read: int = 2, columns=None, verbose=False) -> pd.DataFrame:
    """
    читает num_parts_to_read партиций, преобразует их к pd.DataFrame и возвращает
    :param path_to_dataset: путь до директории с партициями
    :param start_from: номер партиции, с которой начать чтение
    :param num_parts_to_read: количество партиций, которые требуется прочитать
    :param columns: список колонок, которые нужно прочитать из партиции
    :return: pd.DataFrame
    """

    res = []
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset) 
                              if filename.startswith('part')])
    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        chunk = pd.read_parquet(chunk_path,columns=columns)
        res.append(chunk)
    return pd.concat(res).reset_index(drop=True)

from typing import Dict

import numpy as np
import pandas as pd
import pickle
from tqdm.notebook import tqdm

features = ['currency', 'operation_kind', 'card_type', 'operation_type', 'operation_type_group', 'ecommerce_flag',
            'payment_system', 'income_flag', 'mcc', 'country', 'city', 'mcc_category', 'day_of_week',
            'hour', 'weekofyear', 'amnt', 'days_before', 'hour_diff']

embedding_projection = {'currency': (11, 6),
                        'operation_kind': (7, 5),
                        'card_type': (175, 29),
                        'operation_type': (22, 9),
                        'operation_type_group': (4, 3),
                        'ecommerce_flag': (3, 3),
                        'payment_system': (7, 5),
                        'income_flag': (3, 3),
                        'mcc': (108, 22),
                        'country': (24, 9),
                        'city': (163, 28),
                        'mcc_category': (28, 10),
                        'day_of_week': (7, 5),
                        'hour': (24, 9),
                        'weekofyear': (53, 15),
                        'amnt': (10, 6),
                        'days_before': (23, 9),
                        'hour_diff': (10, 6)}


def pad_sequence(array, max_len) -> np.array:
    """
    принимает список списков (array) и делает padding каждого вложенного списка до max_len
    :param array: список списков
    :param max_len: максимальная длина до которой нужно сделать padding
    :return: np.array после padding каждого вложенного списка до одинаковой длины
    """
    add_zeros = max_len - len(array[0])
    return np.array([list(x) + [0] * add_zeros for x in array])


def truncate(x, num_last_transactions=750):
    return x.values.transpose()[:, -num_last_transactions:].tolist()


def transform_transactions_to_sequences(transactions_frame: pd.DataFrame,
                                        num_last_transactions=750) -> pd.DataFrame:
    """
    принимает frame с транзакциями клиентов, сортирует транзакции по клиентам
    (внутри клиента сортирует транзакции по возрастанию), берет num_last_transactions танзакций,
    возвращает новый pd.DataFrame с двумя колонками: app_id и sequences.
    каждое значение в колонке sequences - это список списков.
    каждый список - значение одного конкретного признака во всех клиентских транзакциях.
    Всего признаков len(features), поэтому будет len(features) списков.
    Данная функция крайне полезна для подготовки датасета для работы с нейронными сетями.
    :param transactions_frame: фрейм с транзакциями клиентов
    :param num_last_transactions: количество транзакций клиента, которые будут рассмотрены
    :return: pd.DataFrame из двух колонок (app_id, sequences)
    """
    return transactions_frame \
        .sort_values(['app_id', 'transaction_number']) \
        .groupby(['app_id'])[features] \
        .apply(lambda x: truncate(x, num_last_transactions=num_last_transactions)) \
        .reset_index().rename(columns={0: 'sequences'})


def create_padded_buckets(frame_of_sequences: pd.DataFrame, bucket_info: Dict[int, int],
                          save_to_file_path=None, has_target=True):
    """
    Функция реализует sequence_bucketing технику для обучения нейронных сетей.
    Принимает на вход frame_of_sequences (результат работы функции transform_transactions_to_sequences),
    словарь bucket_info, где для последовательности каждой длины указано, до какой максимальной длины нужно делать
    padding, далее группирует транзакции по бакетам (на основе длины), делает padding транзакций и сохраняет результат
    в pickle файл, если нужно
    :param frame_of_sequences: pd.DataFrame c транзакциями (результат применения transform_transactions_to_sequences)
    :param bucket_info: словарь, где для последовательности каждой длины указано, до какой максимальной длины нужно делать
    padding
    :param save_to_file_path: опциональный путь до файла, куда нужно сохранить результат
    :param has_target: флаг, есть ли в frame_of_sequences целевая переменная или нет. Если есть, то
    будет записано в результат
    :return: возвращает словарь с следюущими ключами (padded_sequences, targets, app_id, products)
    """
    frame_of_sequences['bucket_idx'] = frame_of_sequences.sequence_length.map(bucket_info)
    padded_seq = []
    targets = []
    app_ids = []
    products = []

    for size, bucket in tqdm(frame_of_sequences.groupby('bucket_idx'), desc='Extracting buckets'):
        padded_sequences = bucket.sequences.apply(lambda x: pad_sequence(x, size)).values
        padded_sequences = np.array([np.array(x) for x in padded_sequences])
        padded_seq.append(padded_sequences)

        if has_target:
            targets.append(bucket.flag.values)

        app_ids.append(bucket.app_id.values)
        products.append(bucket['product'].values)

    frame_of_sequences.drop(columns=['bucket_idx'], inplace=True)

    dict_result = {
        'padded_sequences': np.array(padded_seq),
        'targets': np.array(targets) if targets else [],
        'app_id': np.array(app_ids),
        'products': np.array(products),
    }

    if save_to_file_path:
        with open(save_to_file_path, 'wb') as f:
            pickle.dump(dict_result, f)
    return dict_result

import numpy as np
import pickle
import torch

transaction_features = ['currency', 'operation_kind', 'card_type', 'operation_type',
                        'operation_type_group', 'ecommerce_flag', 'payment_system',
                        'income_flag', 'mcc', 'country', 'city', 'mcc_category',
                        'day_of_week', 'hour', 'weekofyear', 'amnt', 'days_before', 'hour_diff']


def batches_generator(list_of_paths, batch_size=32, shuffle=False, is_infinite=False,
                      verbose=False, device=None, output_format='torch', is_train=True):
    """
    функция для создания батчей на вход для нейронной сети для моделей на keras и pytorch.
    так же может использоваться как функция на стадии инференса
    :param list_of_paths: путь до директории с предобработанными последовательностями
    :param batch_size: размер батча
    :param shuffle: флаг, если True, то перемешивает list_of_paths и так же
    перемешивает последовательности внутри файла
    :param is_infinite: флаг, если True,  то создает бесконечный генератор батчей
    :param verbose: флаг, если True, то печатает текущий обрабатываемый файл
    :param device: device на который положить данные, если работа на торче
    :param output_format: допустимые варианты ['tf', 'torch']. Если 'torch', то возвращает словарь,
    где ключи - батчи из признаков, таргетов и app_id. Если 'tf', то возвращает картеж: лист input-ов
    для модели, и список таргетов.
    :param is_train: флаг, Если True, то для кераса вернет (X, y), где X - input-ы в модель, а y - таргеты, 
    если False, то в y будут app_id; для torch вернет словарь с ключами на device.
    :return: бачт из последовательностей и таргетов (или app_id)
    """
    while True:
        if shuffle:
            np.random.shuffle(list_of_paths)

        for path in list_of_paths:
            if verbose:
                print(f'reading {path}')

            with open(path, 'rb') as f:
                data = pickle.load(f)
            padded_sequences, targets, products = data['padded_sequences'], data['targets'], data[
                'products']
            app_ids = data['app_id']
            indices = np.arange(len(products))

            if shuffle:
                np.random.shuffle(indices)
                padded_sequences = padded_sequences[indices]
                targets = targets[indices]
                products = products[indices]
                app_ids = app_ids[indices]

            for idx in range(len(products)):
                bucket, product = padded_sequences[idx], products[idx]
                app_id = app_ids[idx]
                
                if is_train:
                    target = targets[idx]
                
                for jdx in range(0, len(bucket), batch_size):
                    batch_sequences = bucket[jdx: jdx + batch_size]
                    if is_train:
                        batch_targets = target[jdx: jdx + batch_size]
                    
                    batch_products = product[jdx: jdx + batch_size]
                    batch_app_ids = app_id[jdx: jdx + batch_size]
                    
                    if output_format == 'tf':
                        batch_sequences = [batch_sequences[:, i] for i in
                                           range(len(transaction_features))]
                        
                        # append product as input to tf model
                        batch_sequences.append(batch_products)
                        if is_train:
                            yield batch_sequences, batch_targets
                        else:
                             yield batch_sequences, batch_app_ids
                    else:
                        batch_sequences = [torch.LongTensor(batch_sequences[:, i]).to(device)
                                           for i in range(len(transaction_features))]
                        if is_train:
                            yield dict(transactions_features=batch_sequences,
                                       product=torch.LongTensor(batch_products).to(device),
                                       label=torch.LongTensor(batch_targets).to(device),
                                       app_id=batch_app_ids)
                        else:
                            yield dict(transactions_features=batch_sequences,
                                       product=torch.LongTensor(batch_products).to(device),
                                       app_id=batch_app_ids)
        if not is_infinite:
            break


In [78]:
# from utils import read_parquet_dataset_from_local
# from dataset_preprocessing_utils import transform_transactions_to_sequences, create_padded_buckets

In [79]:
import pickle

with open('../constants/buckets_info.pkl', 'rb') as f:
    mapping_seq_len_to_padded_len = pickle.load(f)
    
with open('../constants/dense_features_buckets.pkl', 'rb') as f:
    dense_features_buckets = pickle.load(f)

* Функция `create_buckets_from_transactions` ниже реализует следующий набор действий:
    * Читает `num_parts_to_preprocess_at_once` частей датасета в память
    * Преобразует вещественные и численные признаки к категориальным (используя `np.digitize` и подготовленные бины)
    * Формирует фрейм с транзакциями в виде последовательностей с помощью `transform_transactions_to_sequences`.
    * Если указан `frame_with_ids`, то использует `app_id` из `frame_with_ids` - актуально, чтобы выделить валидационную выборку.
    * Реализует технику `sequence_bucketing` и сохраняет словарь обработанных последовательностей в `.pkl` файл

In [80]:
def create_buckets_from_transactions(path_to_dataset, save_to_path, frame_with_ids = None, 
                                     num_parts_to_preprocess_at_once: int = 1, 
                                     num_parts_total=50, has_target=False):
    block = 0
    for step in tqdm(range(0, num_parts_total, num_parts_to_preprocess_at_once), 
                                   desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(path_to_dataset, step, 
                                                             num_parts_to_preprocess_at_once, 
                                                             verbose=True)
        for dense_col in ['amnt', 'days_before', 'hour_diff']:
            transactions_frame[dense_col] = np.digitize(transactions_frame[dense_col], bins=dense_features_buckets[dense_col])
            
        seq = transform_transactions_to_sequences(transactions_frame)
        seq['sequence_length'] = seq.sequences.apply(lambda x: len(x[1]))
        
        if frame_with_ids is not None:
            seq = seq.merge(frame_with_ids, on='app_id')

        block_as_str = str(block)
        if len(block_as_str) == 1:
            block_as_str = '00' + block_as_str
        else:
            block_as_str = '0' + block_as_str
            
        processed_fragment =  create_padded_buckets(seq, mapping_seq_len_to_padded_len, has_target=has_target, 
                                                    save_to_file_path=os.path.join(save_to_path, 
                                                                                   f'processed_chunk_{block_as_str}.pkl'))
        block += 1

* Разобьем имеющиеся данные на `train` и `val` части. Воспользуемся самым простым способом - для валидации используем 10% случайных данных

In [81]:
train, val = train_test_split(target_frame, random_state=42, test_size=0.1)
train.shape, val.shape

((867429, 3), (96382, 3))

save val buckets

In [82]:
# ! rm -r ../../../tiny_val_buckets
# ! mkdir ../../../tiny_val_buckets

In [83]:
# create_buckets_from_transactions(TRAIN_TRANSACTIONS_PATH, 
#                                 save_to_path='../../../tiny_val_buckets',
#                                 frame_with_ids=val, num_parts_to_preprocess_at_once=1, num_parts_total=500, has_target=True)

In [84]:
# ! rm -r ../../../val_buckets
# ! mkdir ../../../val_buckets

In [85]:
# create_buckets_from_transactions(TRAIN_TRANSACTIONS_PATH, 
#                                 save_to_path='../../../val_buckets',
#                                 frame_with_ids=val, num_parts_to_preprocess_at_once=1, num_parts_total=50, has_target=True)

In [86]:
# path_to_dataset = '../../../val_buckets'
path_to_dataset = '../../../tiny_val_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_val = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_val

['../../../tiny_val_buckets/processed_chunk_000.pkl',
 '../../../tiny_val_buckets/processed_chunk_001.pkl',
 '../../../tiny_val_buckets/processed_chunk_002.pkl',
 '../../../tiny_val_buckets/processed_chunk_003.pkl',
 '../../../tiny_val_buckets/processed_chunk_004.pkl']

save train buckets

In [87]:
# ! rm -r ../../../tiny_train_buckets
# ! mkdir ../../../tiny_train_buckets

In [88]:
# TRAIN_TRANSACTIONS_PATH


In [89]:
# create_buckets_from_transactions(TRAIN_TRANSACTIONS_PATH, 
#                                 save_to_path='../../../tiny_train_buckets',
#                                 frame_with_ids=train, num_parts_to_preprocess_at_once=1, num_parts_total=500, has_target=True)

In [90]:
# ! rm -r ../../../train_buckets
# ! mkdir ../../../train_buckets

In [91]:
# create_buckets_from_transactions(TRAIN_TRANSACTIONS_PATH, 
#                                 save_to_path='../../../train_buckets',
#                                 frame_with_ids=train, num_parts_to_preprocess_at_once=1, num_parts_total=50, has_target=True)

In [92]:
path_to_dataset = '../../../train_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_train = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
# dataset_train

### 2. Modeling

In [93]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


* Для создания модели будем использовать фреймворк `torch`. В нем есть все, чтобы писать произвольные сложные архитектуры и быстро эксперементировать. Для того, чтобы мониторить и логировать весь процесс во время обучения сетей, рекомендуется использовать надстройки над данным фреймворков, например, `lightning`.

* В бейзлайне мы предлагаем базовые компоненты, чтобы можно было обучать нейронную сеть и отслеживать ее качество. Для этого вам предоставлены следующие функции:
    * `data_generators.batches_generator` - функция-генератор, итеративно возвращает батчи, поддерживает батчи для `tensorflow.keras` и `torch.nn.module` моделей. В зависимости от флага `is_train` может быть использована для генерации батчей на train/val/test стадию.
    * функция `pytorch_training.train_epoch` - обучает модель одну эпоху.
    * функция `pytorch_training.eval_model` - проверяет качество модели на отложенной выборке и возвращает roc_auc_score.
    * функция `pytorch_training.inference` - делает предикты на новых данных и готовит фрейм для проверяющей системы.
    * класс `training_aux.EarlyStopping` - реализует early_stopping, сохраняя лучшую модель. Пример использования приведен ниже.

In [94]:
# from data_generators import batches_generator, transaction_features
# from pytorch_training import train_epoch, eval_model, inference
# from training_aux import EarlyStopping#

* Все признаки в нашей модели будут категориальными. Для их представления в модели используем категориальные эмбеддинги. Для этого нужно каждому категориальному признаку задать размерность латентного пространства. Используем [формулу](https://forums.fast.ai/t/size-of-embedding-for-categorical-variables/42608) из библиотеки `fast.ai`. Все отображения хранятся в файле `embedding_projections.pkl`

In [95]:
with open('../constants/embedding_projections.pkl', 'rb') as f:
    embedding_projections = pickle.load(f)
# embedding_projections

* Реализуем модель. Все входные признаки представим в виде эмбеддингов, сконкатенируем, чтобы получить векторное представление транзакции. Подадим последовательности в `GRU` рекуррентную сеть. Используем последнее скрытое состояние в качестве выхода сети. Представим признак `product` в виде отдельного эмбеддинга. Сконкатенируем его с выходом сети. На основе такого входа построим небольшой `MLP`, выступающий классификатором для целевой задачи. Используем градиентный спуск, чтобы решить оптимизационную задачу. 

In [96]:
class TransactionsRnn(nn.Module):
    def __init__(self, transactions_cat_features, embedding_projections, product_col_name='product', rnn_units=128, top_classifier_units=32):
        super(TransactionsRnn, self).__init__()
        self._transaction_cat_embeddings = nn.ModuleList([self._create_embedding_projection(*embedding_projections[feature]) 
                                                          for feature in transactions_cat_features])
                
        self._product_embedding = self._create_embedding_projection(*embedding_projections[product_col_name], padding_idx=None)
        
        self._gru = nn.GRU(input_size=sum([embedding_projections[x][1] for x in transactions_cat_features]),
                             hidden_size=rnn_units, batch_first=True, bidirectional=False)
        
        self._hidden_size = rnn_units
                
        self._top_classifier = nn.Linear(in_features=rnn_units+embedding_projections[product_col_name][1], 
                                         out_features=top_classifier_units)
        self._intermediate_activation = nn.ReLU()
        
        self._head = nn.Linear(in_features=top_classifier_units, out_features=1)
    
    def forward(self, transactions_cat_features, product_feature):
        batch_size = product_feature.shape[0]
        
        embeddings = [embedding(transactions_cat_features[i]) for i, embedding in enumerate(self._transaction_cat_embeddings)]
        concated_embeddings = torch.cat(embeddings, dim=-1)
        
        _, last_hidden = self._gru(concated_embeddings)
        last_hidden = torch.reshape(last_hidden.permute(1, 2, 0), shape=(batch_size, self._hidden_size))
        
        product_embed = self._product_embedding(product_feature)
        
        intermediate_concat = torch.cat([last_hidden, product_embed], dim=-1)
                
        classification_hidden = self._top_classifier(intermediate_concat)
        activation = self._intermediate_activation(classification_hidden)
        
        logit = self._head(activation)
        
        return logit
    
    @classmethod
    def _create_embedding_projection(cls, cardinality, embed_size, add_missing=True, padding_idx=0):
        add_missing = 1 if add_missing else 0
        return nn.Embedding(num_embeddings=cardinality+add_missing, embedding_dim=embed_size, padding_idx=padding_idx)
    
def get_model():
    return TransactionsRnn(transaction_features, embedding_projections,
                                            rnn_units=4, top_classifier_units=4).to(device)


### 3. Training

In [97]:
# for

In [98]:
! mkdir ../../rnn_baseline/checkpoints/

mkdir: cannot create directory ‘../../rnn_baseline/checkpoints/’: File exists


In [99]:
! rm -r ../../rnn_baseline/checkpoints/pytorch_baseline
! mkdir ../../rnn_baseline/checkpoints/pytorch_baseline

* Для того, чтобы детектировать переобучение используем EarlyStopping.

In [100]:
# path_to_checkpoints = '../../rnn_baseline/checkpoints/pytorch_baseline/'
# es = EarlyStopping(patience=3, mode='max', verbose=True, 
#                    save_path=os.path.join(path_to_checkpoints, 'best_checkpoint.pt'), 
#                    metric_name='ROC-AUC', save_format='torch')

In [101]:
CLIENTS_CNT = 3
# num_epochs =5#15
# train_batch_size = 128
# val_batch_szie = 128
train_batch_size = val_batch_szie = 128

In [102]:
model = get_model() #TransactionsRnn(transaction_features, embedding_projections, rnn_units=4, top_classifier_units=4 ).to(device)

In [103]:
optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())

* Запустим цикл обучения, каждую эпоху будем логировать лосс, а так же roc-auc на валидации и на обучении. Будем сохрнаять веса после каждой эпохи, а так же лучшие с помощью early_stopping.

In [104]:
file_num_constarint = 1
dataset_val = dataset_val#[:file_num_constarint]
dataset_train = dataset_train#[:file_num_constarint]

In [105]:
trains = [[x] for x in dataset_train]
vals = [[x] for x in dataset_val]

### fedavg

In [106]:
#Zexi Li
def fedavg(models, list_nums_local_data):
    # feat_cnt= dict(models[0][0].named_parameters())['weight'].data.shape[1]
    parameters = [m.state_dict() for m in models]
    # parameters is a list of state_dicts
    fedavg_global_params = copy.deepcopy(parameters[0])
    for name_param in parameters[0]:
        list_values_param = []
        for dict_local_params, num_local_data in zip(parameters, list_nums_local_data):
            list_values_param.append(dict_local_params[name_param] * num_local_data)
        value_global_param = sum(list_values_param) / sum(list_nums_local_data)
        fedavg_global_params[name_param] = value_global_param
    # return fedavg_global_params
    res = get_model()
    res.load_state_dict(fedavg_global_params)
    return res

## Trainer

In [107]:
class Trainer:
    def __init__(self, dataset_train, dataset_val, model, dataset_name) -> None:
        self.model = model
        self.dataset_name = dataset_name
        self.loss_function = nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())
        self.dataset_train = dataset_train
        self.dataset_val = dataset_val

    def train_one_epoch(self, batch_size=64, shuffle=True,
                        print_loss_every_n_batches=500, device=None, n_batches_to_train=None, verbose=False):
        """
        делает одну эпоху обучения модели, логирует
        :param model: nn.Module модель
        :param optimizer: nn.optim оптимизатор
        :param dataset_train: путь до директории с последовательностями
        :param batch_size: размерм батча
        :param shuffle: флаг, если True, то перемешивает данные
        :param print_loss_every_n_batches: число батчей после которых логируется лосс на этих батчах
        :param device: device, на который будут положены данные внутри батча
        :return: None
        """
        train_generator = batches_generator(self.dataset_train, batch_size=batch_size, shuffle=shuffle,
                                            device=device, is_train=True, output_format='torch')
        # loss_function = nn.BCEWithLogitsLoss()

        num_batches = 1
        running_loss = 0.0

        self.model.train()

        for batch in tqdm(train_generator, 
                        #   desc=f'Training {self.dataset_train}'
                          ):

            output = torch.flatten(self.model(
                batch['transactions_features'], batch['product']))

            batch_loss = self.loss_function(output, batch['label'].float())

            batch_loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            running_loss += batch_loss

            if num_batches % print_loss_every_n_batches == 0:
                print(
                    f'Training loss after {num_batches} batches: {running_loss / num_batches}', end='\r')

            num_batches += 1

            if n_batches_to_train is not None and num_batches >= n_batches_to_train:
                if verbose:
                    print(f'Reached n_batches_to_train = {n_batches_to_train}')
                break

        if verbose:
            print(
            f'Training loss after epoch: {running_loss / num_batches}', end='\r')

    def eval_model(self, batch_size=32, device=None, custom_model=None,
                   n_batches_to_val=None, verbose=False) -> float:
        """
        функция для оценки качества модели на отложенной выборке, возвращает roc-auc на валидационной
        выборке
        :param model: nn.Module модель
        :param dataset_val: путь до директории с последовательностями
        :param batch_size: размер батча
        :param device: device, на который будут положены данные внутри батча
        :return: val roc-auc score
        """
        preds = []
        targets = []
        val_generator = batches_generator(self.dataset_val, batch_size=batch_size, shuffle=False,
                                          device=device, is_train=True, output_format='torch')
        
        model = custom_model if custom_model is not None else self.model

        model.eval()
        num_batches = 0
        # for batch in tqdm(val_generator, desc='Evaluating model' if verbose else None):
        for batch in val_generator:
            targets.extend(batch['label'].detach().cpu().numpy().flatten())
            output = model(batch['transactions_features'], batch['product'])
            preds.extend(output.detach().cpu().numpy().flatten())
            num_batches+=1
        
            if n_batches_to_val is not None and num_batches>n_batches_to_val:
                if verbose:
                    print(f'Reached n_batches_to_val = {n_batches_to_val}')
                break

        return roc_auc_score(targets, preds)


trainers = []
for i in range(CLIENTS_CNT):
    trainers.append(Trainer(
        dataset_train[5*i:5*(i+1)],
                             dataset_val[:2],#TODO Note! Validation on the same set
                            get_model(), f"ds {i}"
                            ))

## Sync train

In [108]:
colors = list(plotly.colors.DEFAULT_PLOTLY_COLORS)
color_by_dataset={}#
for i,t in enumerate(trainers):
    color_by_dataset[t.dataset_name] = colors[i]

def get_line_mode(avg):
    return {} if avg else {"dash": "dot"}

In [109]:
num_epochs = 20

In [110]:
metrics = {}
# VAL_ON = 25
for m in ['local', 'avg']:
    for ds in [t.dataset_name for t in trainers]:
        metrics[(m, ds)] = []
for epoch in tqdm(range(num_epochs)):
    fig = go.Figure()
    fig.update_layout(
        title_text=f"Test metrics during training. Epoch {epoch+1} completed.")
    for t in trainers:
        t: Trainer
        t.train_one_epoch(batch_size=train_batch_size,
                          shuffle=True, print_loss_every_n_batches=10**10, device=device, 
                          n_batches_to_train=500)
        val_metric = t.eval_model(
            batch_size=val_batch_szie, device=device, n_batches_to_val=None)
        metrics[('local', t.dataset_name)].append(val_metric)
        s = pd.Series(metrics[('local', t.dataset_name)])
        fig.add_trace(go.Scatter(
            x=s.index,
            y=s,
            line=get_line_mode(False) | {
                'color': color_by_dataset[t.dataset_name]},
            name=f"local {t.dataset_name}"
        ))

    avg_model = fedavg([t.model for t in trainers],
                       [1]*len(trainers))  # mock
    for t in trainers:
        val_metric = t.eval_model(
            batch_size=val_batch_szie, device=device, custom_model=avg_model, n_batches_to_val=None)
        metrics[('avg', t.dataset_name)].append(val_metric)
        s = pd.Series(metrics[('avg', t.dataset_name)])
        fig.add_trace(go.Scatter(x=s.index,
                                 y=s,
                                 line=get_line_mode(True) | {
                                     'color': color_by_dataset[t.dataset_name]},
                                 name=f"avg on {t.dataset_name}"))

    # print(f'Epoch {epoch+1} completed. Train roc-auc: {train_roc_auc}, Val roc-auc: {val_roc_auc}')

    display(Image(fig.to_image(format="png")))

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [ ]:
colors = list(plotly.colors.DEFAULT_PLOTLY_COLORS)
color_by_dataset={}#
for i,t in enumerate(trainers):
    color_by_dataset[t.dataset_name] = colors[i]
def get_line_mode(avg):
    return {} if avg else {"dash": "dot"}

fig = go.Figure()
fig.update_layout(title_text=f"Test metrics during training")
for i, ((stage, dataset), m) in enumerate(metrics.items()):
    s = pd.Series(m)
    d = "dot" if dataset=='local' else 'solid'
    fig.add_trace(go.Scatter(x=s.index, y=s,line=get_line_mode(stage=='avg')|{'color':color_by_dataset[dataset]},name=f"{stage} {dataset}")
                )
fig.show()

In [ ]:
for t in trainers:
        val_metric = t.eval_model(batch_size=val_batch_szie, device=device, custom_model=avg_model, n_batches_to_val=10**10)
        print(val_metric)

0.6139582718417945
0.6139582718417945
0.6139582718417945


In [ ]:
# for epoch in range(0):
#     print(f'Starting epoch {epoch+1}')
#     train_epoch(model, optimizer, dataset_train, batch_size=train_batch_size, 
#                 shuffle=True, print_loss_every_n_batches=25, device=device, n_batches_to_train= 101)
    
#     val_roc_auc = eval_model(model, dataset_val, batch_size=val_batch_szie, device=device)
#     es(val_roc_auc, model)
    
#     if es.early_stop:
#         print('Early stopping reached. Stop training...')
#         break
#     torch.save(model.state_dict(), os.path.join(path_to_checkpoints, f'epoch_{epoch+1}_val_{val_roc_auc:.3f}.pt'))
    
#     train_roc_auc = eval_model(model, dataset_train, batch_size=val_batch_szie, device=device)
#     print(f'Epoch {epoch+1} completed. Train roc-auc: {train_roc_auc}, Val roc-auc: {val_roc_auc}')

### 4. Submission

* Все готово, чтобы сделать предсказания для тестовой выборки. Нужно только подготовить данные в том же формате, как и для train.

In [ ]:
! rm -r ../../../test_buckets
! mkdir ../../../test_buckets

In [ ]:
test_frame = pd.read_csv('../../../test_target_contest.csv')
test_frame.head()

,app_id,product
0,1063620,0
1,1063621,0
2,1063622,1
3,1063623,1
4,1063624,2


In [ ]:
TRAIN_TRANSACTIONS_PATH

'../../../train_transactions_contest/'

In [ ]:
TEST_TRANSACTIONS_PATH

'../../../test_transactions_contest/'

In [ ]:
# create_buckets_from_transactions(TEST_TRANSACTIONS_PATH, 
#                                 save_to_path='../../../test_buckets', frame_with_ids=test_frame, 
#                                  num_parts_to_preprocess_at_once=10, num_parts_total=50, has_target=False)

In [ ]:
path_to_test_dataset = '../../../test_buckets/'
dir_with_test_datasets = os.listdir(path_to_test_dataset)
dataset_test = sorted([os.path.join(path_to_test_dataset, x) for x in dir_with_test_datasets])

dataset_test

[]

* Отдельный вопрос, какую из построенных моделей использовать для того, чтобы делать предсказания на тест. Можно выбирать лучшую по early_stopping. В таком случае есть риск, что мы подгонимся под валидационную выборку, особенно если она не является очень репрезентативной, однако это самый базовый вариант (используем его). Можно делать разные версии ансамблирования, используя веса с разных эпох. Такой подход требует дополнительного кода (обязательно попробуйте его!). Наконец, можно выбирать такую модель, которая показывает хорошие результаты на валидации и в то же время, не слишком переучена под train выборку.

In [ ]:
! ls $path_to_checkpoints

In [ ]:
model.load_state_dict(torch.load(os.path.join(path_to_checkpoints, 'best_checkpoint.pt')))

FileNotFoundError: [Errno 2] No such file or directory: '../../rnn_baseline/checkpoints/pytorch_baseline/best_checkpoint.pt'

In [ ]:
test_preds = inference(model, dataset_test, batch_size=128, device=device)

In [ ]:
test_preds.head()

,app_id,score
0,1063655,-3.865098
1,1063672,-2.472911
2,1063694,-3.957081
3,1063709,-3.381659
4,1063715,-4.051003


In [ ]:
# test_preds.to_csv('rnn_baseline_submission.csv', index=None) # ~ 0.750 на public test